# Overview

Before we got our own fine-tuned LLMs. How we can make our basline LLM in a higher accuracy? One of the answers might be **Retrieval Augmented Generation(RAG)**. RAG enables LLMs reach beyond their static knowledge and grab real-time information quite easily. RAG taps into external sources, like internal company documents, to understand the context of our queries a bit better.

![](https://cdn.masto.host/sigmoidsocial/media_attachments/files/112/064/408/832/155/132/small/3f7a8bd57563298f.webp)

In this notebook, we are going to use RAG model. Retrieval-agumented generation("RAG") models combine the powers of pretrained dense retrieval(DPR) and Seq2Seq2 models. RAG models retrieve docs, pass them to a seq2seq2 model, then marginalize to generate outputs. The retriever and seq2seq modules are initlized from pretrained models, and fine-tuned jointly, allowing both retreival and generation to adapt to downstream tasks. Here are some classes inside transformers that we need to know:


# RagConfig Class

It stores the configuration of a RagModel which is used to control thr model outputs.


# Retriever Class

It used to get documents from vector queries. It retrives the documents embeddings as well as the documents contents, and it formats them to be used with a RagModel.


# RAGModel Class

RAG is a seq2seq model which encapsulates two core components: a question encoder and a generator. During a forward pass, we encode the input with the question encoder and pass it to the retriever to extract releveant context documents. The documents are then prepended to the input. Such contextualized inputs is passed to the generator.

The question encoder can be any autoencoding model, preferably **DPPQuestionEncoder** and the generator can be any **seq2seq**, preferably **BartForConditionalGeneration**. So, we can initialzed with the RAG model with any autoencoding mdoel as the question_encoder and any seq2seq model with language model heads as the generator. It inherits from **PreTrainedModel**, and it is also a PyTorch **torch.nn.Module** subclass. 

In [16]:
%%capture
!pip install transformers==4.38.2
# !pip install accelerate==0.27.2
!pip install datasets==2.18.0
# !pip install peft==0.9.0
# !pip install bitsandbytes==0.42.0

In [10]:
import os
import torch
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))


os.environ["MODEL_NAME"] = "facebook/rag-token-base"
os.environ["DATASET"] = ""

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic=True
    # https://github.com/huggingface/transformers/issues/28731
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    device='cuda'
else:
    device='cpu'

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0


In [4]:
!python --version

Python 3.10.13


In [6]:
!conda env list

# conda environments:
#
base                     /opt/conda



In [ ]:
!conda install -c pytorch faiss-cpu=1.8.0 -y

In [8]:
# !conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.1.2



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu=1.8.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    faiss-gpu-1.8.0            |py3.10_h4c7d538_0_cuda12.1.1         5.0 MB  pytorch
    libfaiss-1.8.0             |h046e95b_0_cuda12.1.1       268.9 MB  pytorch
    ------------------------------------------------------------
                                           Total:       273.8 MB

The following NEW packages will be INSTALLED:

  faiss-gpu          pytorch/linux-64::faiss-gpu-1.8.0-py3.10_h4c7d538_0_cuda12.1.1 
  l

In [12]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("MODEL_NAME"))
tokenizer

config.json:   0%|          | 0.00/4.55k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.


(…)enerator_tokenizer/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [17]:
from transformers import RagRetriever

retriever=RagRetriever.from_pretrained(os.getenv("MODEL_NAME"), index_name="exact", use_dummy_dataset=True)
retriever

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
engine=RagModel.from_pretrained(os.getenv("MODEL_NAME"), retriever=retriever, device=device)
engine

In [ ]:
token_input=tokenizer("How many poeple live in Melbourne?", return_tensors="pt")
print(token_input)

In [ ]:
outputs=engine(input_ids=token_input["input_ids"])

# Reference List

* https://www.promptingguide.ai/research/rag
* https://blog.gopenai.com/retrieval-augmented-generation-rag-585aa903d6bd